In [79]:
import requests
#TODO: add kinopoisk similar movies to movies.csv

In [91]:
import json
import pandas as pd
import re

def json_names_list_to_list(json_list):
    list = []
    for json in json_list:
        list.append(json["name"])
    return list

def replace_nbsp(description):
    return re.sub(r"\s", " ", description)

movies_df = None
for i in range (1, 5):
    movies_json_file = open(f"movies{i}.json", "r", encoding="utf-8")
    movies_json = json.load(movies_json_file)["docs"]
    movies = pd.DataFrame()
    movies.index = [movie["id"] for movie in movies_json]
    movies["type"] = [movie["type"] for movie in movies_json]
    movies["name"] = [movie["name"] for movie in movies_json]
    movies["age_rating"] = [movie["ageRating"] for movie in movies_json]
    movies["top250"] = [movie["top250"] for movie in movies_json]
    movies["short_description"] = [movie["shortDescription"] for movie in movies_json]
    movies["description"] = [replace_nbsp(movie["description"]) for movie in movies_json]
    movies["year"] = [movie["year"] for movie in movies_json]
    movies["genres"] = [json_names_list_to_list(movie["genres"]) for movie in movies_json]
    movies["countries"] = [json_names_list_to_list(movie["countries"]) for movie in movies_json]
    movies["movie_length_minutes"] = [movie["movieLength"] for movie in movies_json]
    movies["kinopoisk_rating"] = [movie["rating"]["kp"] for movie in movies_json]
    movies["imdb_rating"] = [movie["rating"]["imdb"] for movie in movies_json]
    movies["world_film_critics_rating"] = [movie["rating"]["filmCritics"] for movie in movies_json]
    movies["russian_film_critics_rating"] = [movie["rating"]["russianFilmCritics"] for movie in movies_json]
    movies["reviews"] = [[] for movie in movies_json]
    movies_df = pd.concat([movies_df, movies])
movies_df.to_csv("movies.csv", encoding="utf-8")
movies_df

,type,name,age_rating,top250,short_description,description,year,genres,countries,movie_length_minutes,kinopoisk_rating,imdb_rating,world_film_critics_rating,russian_film_critics_rating,reviews
535341,movie,1+1,18.0,4.0,Аристократ на коляске нанимает в сиделки бывше...,"Пострадав в результате несчастного случая, бог...",2011,"[драма, комедия, биография]",[Франция],112.0,8.816,8.5,6.8,100.0000,[]
1143242,movie,Джентльмены,18.0,33.0,"Наркобарон хочет уйти на покой, но криминальны...",Один ушлый американец ещё со студенческих лет ...,2019,"[криминал, комедия, боевик]","[США, Великобритания]",113.0,8.563,7.8,6.5,86.3636,[]
462682,movie,Волк с Уолл-стрит,18.0,72.0,Восхождение циника-гедониста на бизнес-олимп 1...,1987 год. Джордан Белфорт становится брокером ...,2013,"[драма, криминал, биография, комедия]",[США],180.0,7.998,8.2,7.8,77.4194,[]
41519,movie,Брат,18.0,48.0,Дембель Данила Багров защищает слабых в Петерб...,"Демобилизовавшись, Данила Багров вернулся в ро...",1997,"[драма, криминал, боевик]",[Россия],100.0,8.307,7.8,7.6,0.0000,[]
1318972,movie,Гнев человеческий,18.0,NaN,"Хмурый мужчина прикидывается инкассатором, что...",Грузовики лос-анджелесской инкассаторской комп...,2021,"[боевик, триллер]","[Великобритания, США]",119.0,7.607,7.1,6.3,72.9730,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,cartoon,История игрушек 2,0.0,NaN,Ковбой Вуди в опасности! Только друзья могут е...,Ковбой Вуди и другие игрушки мальчика по имени...,1999,"[мультфильм, фэнтези, комедия, приключения, се...",[США],92.0,7.807,7.9,8.7,0.0000,[]
842673,movie,Бегущий в лабиринте: Испытание огнём,16.0,NaN,Сбежавшие из Лабиринта подростки борются с зом...,"Лабиринт пройден, но Томасу, Терезе, Минхо и п...",2015,"[фантастика, боевик, триллер, приключения]",[США],131.0,6.164,6.3,5.4,83.3333,[]
665315,movie,Другая женщина,16.0,NaN,"Карли, Кейт и Эмбер узнают, что у них — один м...","Любовная связь с женатым мужчиной – это, конеч...",2014,"[мелодрама, комедия]",[США],109.0,6.776,6.0,4.3,33.3333,[]
6345,movie,Час пик,12.0,NaN,Коп-пустомеля Крис Такер и молчаливый профи Дж...,В Лос-Анджелесе злодеи похищают малолетнюю доч...,1998,"[боевик, комедия, криминал]",[США],98.0,7.740,7.0,6.2,0.0000,[]


In [96]:
counter = 0

for movie in movies_df.index:
    movie_reviews_json = requests.get(f"https://api.kinopoisk.dev/v1.4/review?page=1&limit=25&movieId={movie}", headers={"accept": "application/json", "X-API-KEY": "2N7F8ET-HYHM6JX-K45YP02-5G1WWSP"}).json()["docs"]

    movie_reviews_list = []
    for review in movie_reviews_json:
        movie_reviews_list.append(replace_nbsp(review["review"]))
        
    movies_df.at[movie, "reviews"] = movie_reviews_list
    counter += 1
    if counter == 100:
        break

movies_df.to_csv("movies.csv", encoding="utf-8")